In [27]:
%load_ext autoreload
%autoreload 2

from init import ROOT_DIR
from common import kite_spectrum_jax_vec as vkite
import numpy as np
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

import dh5


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
parameters = vkite.ParamType(
    ECs_GHz=0.072472,
    EL_GHz=1.269,
    ECJ_GHz=4.9895,
    EJ_GHz=17.501,
    eps=0.05702,
    ECc_GHz=0.003989,
    f_r_GHz=4.337,
)

dim_kept = 18

In [3]:
# ev, ek, _ = vkite.get_qr_esys_with_jac(
#     dims=vkite.DEFAULT_DIMS,
#     dim_kept=20,
#     n_g=0.0,
#     phiext=np.pi,
#     **parameters,
#     output_jac_params=tuple(),
# )

# expected_readout_number = vkite.evaluate_expected_resonator_number(
#     ek, np.prod(vkite.DEFAULT_DIMS[:3]), vkite.DEFAULT_DIMS[3]
# )

In [ ]:
flux_tabs = np.linspace(0, np.pi, 201)

In [12]:
float(flux_tabs[0])

0.0

In [14]:
# ev, ek, _ = vkite.get_qr_esys_with_jac(
#     dims=vkite.DEFAULT_DIMS,
#     dim_kept=dim_kept,
#     n_g=0.0,
#     phiext=float(flux_tabs[0]),
#     **parameters,
#     output_jac_params=None,
# )

In [20]:
ev_tab = np.zeros((len(flux_tabs), dim_kept))
ek_tab = np.zeros(
    (len(flux_tabs), np.prod(vkite.DEFAULT_DIMS), dim_kept), dtype=jnp.complex128
)
expected_readout_number_tab = np.zeros((len(flux_tabs), dim_kept))
for i, flux in enumerate(tqdm(flux_tabs)):
    ev, ek, _ = vkite.get_qr_esys_with_jac(
        dims=vkite.DEFAULT_DIMS,
        dim_kept=dim_kept,
        n_g=0.0,
        phiext=float(flux),
        **parameters,
        output_jac_params=tuple(),
    )
    ev_tab[i] = ev
    ek_tab[i] = ek
    expected_readout_number_tab[i] = vkite.evaluate_expected_resonator_number(
        ek, np.prod(vkite.DEFAULT_DIMS[:3]), vkite.DEFAULT_DIMS[3]
    )


  0%|          | 0/201 [00:00<?, ?it/s]

In [29]:
data = dh5.DH5(
    ROOT_DIR / "data" / "theory" / "kite_readout_flux_sweep_ev.h5", "w", overwrite=True
)

data["fluxes"] = flux_tabs
data["evs"] = ev_tab
data["expected_readout_numbers"] = expected_readout_number_tab
data.save()


DH5 (w): 
 {
    "evs": "shape: (201, 18) (type: ndarray)",
    "expected_readout_numbers": "shape: (201, 18) (type: ndarray)",
    "fluxes": "variable of type ArrayImpl"
}

In [30]:
data = dh5.DH5(
    ROOT_DIR / "data" / "theory" / "kite_readout_flux_sweep_ek.h5", "w", overwrite=True
)

data["fluxes"] = flux_tabs
data["evs"] = ev_tab
data["eks"] = ek_tab
data["expected_readout_numbers"] = expected_readout_number_tab
data.save()


DH5 (w): 
 {
    "eks": "shape: (201, 97280, 18) (type: ndarray)",
    "evs": "shape: (201, 18) (type: ndarray)",
    "expected_readout_numbers": "shape: (201, 18) (type: ndarray)",
    "fluxes": "variable of type ArrayImpl"
}